In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras import layers,models
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train='../input/tabular-playground-series-oct-2021/train.csv'
test='../input/tabular-playground-series-oct-2021/test.csv'
sample_sub='../input/tabular-playground-series-oct-2021/sample_submission.csv'

In [ ]:
train=pd.read_csv(train)
train.head()

In [ ]:
print('Shape of data ',train.shape)


In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

train=reduce_mem_usage(train)

In [ ]:
train.drop('id',axis=1,inplace=True)
y_train=train['target']
train.drop('target',axis=1,inplace=True)
X_train=train

In [ ]:
test=pd.read_csv(test)
test.drop('id',axis=1,inplace=True)
X_test=test

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.4,stratify=y_train)

In [ ]:
model=models.Sequential()
model.add(layers.Dense(256,activation='relu',input_shape=X_train.iloc[0].shape))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(8,activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='RMSprop',metrics=['acc'])
model.fit(X_train,y_train,epochs=20,batch_size=1028,validation_data=(X_val,y_val))

In [ ]:
pred=model.predict(X_test)

In [ ]:
pred_final=pd.DataFrame({'id':np.arange(1000000,1500000),'target':pred.flatten()})

In [ ]:
pred_final.to_csv('submission.csv',index=False)